In [ ]:
# Experimental Topology
# lockin AO 2 -> Analog Attenuator Vdd
# lockin AO 3 -> Analog Attenuator Vc (Control Voltages)
# Keithley Output -> Voltage divider with 1 MOhm, DAQmx AI1
# AWG Sync Marker Out -> DAQmx PFI0
# AWG Samp. Marker Out -> PSPL Trigger

In [1]:
cd ..

C:\Users\qlab\Documents\GitHub\pycontrol


In [2]:
from pycontrol.filters.io import WriteToHDF5
import numpy as np
import itertools
import time
import matplotlib.pyplot as plt
import analysis.switching as sw
from adapt import refine
from analysis.h5shell import h5shell

In [3]:
from experiments.Pulse_Switching_CSHE import SwitchingExperiment

In [4]:
exp = SwitchingExperiment()
exp.sample = "CSHE5-C2R3"
exp.comment = "Phase Diagram -  P to AP - Field at 8.1mT - Interations = 14 - Settle delay = 200us - Reset 0.7V 5ns"
exp.polarity = -1 # 1: AP to P; -1: P to AP
exp.iteration = 14
exp.field.value = 0.0081
exp.measure_current = 3e-6
exp.reset_amplitude = 0.7
exp.reset_duration  = 5.0e-9
exp.settle_delay    = 200e-6
exp.pspl_atten      = 5
exp.init_instruments()

In [5]:
wr = WriteToHDF5("data\CSHE-Switching\CSHE-Die5-C2R3\CSHE5-C2R3-P2AP_81Oe_2016-07-27.h5")
edges = [(exp.daq_buffer, wr.data)]
exp.set_graph(edges)

In [6]:
coarse_ts = 1e-9*np.linspace(0.1, 10, 10) # List of durations
coarse_vs = np.linspace(0.3, 1.0, 10)
points    = [coarse_ts, coarse_vs]
points    = list(itertools.product(*points))

main_sweep = exp.add_unstructured_sweep([exp.pulse_duration, exp.pulse_voltage], points)

In [7]:
figs = []
t1 = time.time()
for i in range(exp.iteration):
    exp.init_progressbar(1, notebook=True)
    exp.reset()
    exp.run_sweeps()
    points, mean = sw.load_switching_data(wr.filename, threshold=0.36)
    figs.append(sw.phase_diagram_mesh(points, mean, title="Iteration={}".format(i)))
    new_points = refine.refine_scalar_field(points, mean, all_points=False,
                                criterion="integral", threshold = "one_sigma")
    if new_points is None:
        print("No more points can be added.")
        break
    #
    print("Elapsed time: {} min.".format((time.time()-t1)/60))
    main_sweep.update_values(new_points)

t2 = time.time()
exp.shutdown_instruments()
print("Total points: {}".format(len(points)))
print("Total elapsed time: {} min.".format((t2-t1)/60))

28 new points added.
Elapsed time: 1.4301918983459472 min.

52 new points added.
Elapsed time: 1.844086241722107 min.

71 new points added.
Elapsed time: 2.589890130360921 min.

101 new points added.
Elapsed time: 3.598311642805735 min.

184 new points added.
Elapsed time: 5.020888102054596 min.

229 new points added.
Elapsed time: 7.593802392482758 min.

314 new points added.
Elapsed time: 10.820241196950276 min.

507 new points added.
Elapsed time: 15.254851055145263 min.

749 new points added.
Elapsed time: 22.60021251042684 min.

1012 new points added.
Elapsed time: 33.6762815117836 min.

1502 new points added.
Elapsed time: 48.97390834093094 min.

2223 new points added.
Elapsed time: 71.13639059464137 min.

3213 new points added.
Elapsed time: 105.21855354309082 min.

4525 new points added.
Elapsed time: 156.62569984197617 min.
Total points: 10285
Total elapsed time: 156.62571605443955 min.


In [8]:
plt.show()

In [8]:
points, mean = sw.load_switching_data(wr.filename, start_state=0, display=True, threshold=0.36)
fig_sw = sw.phase_diagram_mesh(points, mean)

In [9]:
# Reset failures
points, fails = sw.load_switching_data(wr.filename, failure=True)
fig_fail = sw.phase_diagram_mesh(points, fails)
plt.show()

C:\Users\qlab\Anaconda3\lib\site-packages\sklearn\cluster\k_means_.py:821: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  n_jobs=self.n_jobs)


In [11]:
# For evaluation of adaptive method, plot the mesh
mesh, scale_factors = sw.scaled_Delaunay(points)
fig_mesh = sw.phase_diagram_mesh(points, mean, shading='gouraud')
plt.triplot(mesh.points[:,0]/scale_factors[0],
            mesh.points[:,1]/scale_factors[1], mesh.simplices.copy());
plt.show()

In [ ]:
points2, mean2 = sw.load_switching_data(wr.filename, start_state=1) # threshold = 130e+3*3e-6)
fig2 = sw.phase_diagram_mesh(points2, mean2)
plt.show()

C:\Users\qlab\Anaconda3\lib\site-packages\sklearn\cluster\k_means_.py:821: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  n_jobs=self.n_jobs)


In [10]:
import importlib
importlib.reload(sw)

<module 'analysis.switching' from 'C:\\Users\\qlab\\Documents\\GitHub\\pycontrol\\analysis\\switching.py'>